What is Paket?
Paket is a **dependency manager** for **.NET** and mono projects. It is designed to work well with **NuGet** packages and also enables referencing files directly from **Git repositories** or any **HTTP resource**. 

In [8]:
#load "Paket.fsx"
Paket.Package
  [ "Base58Check -Version 0.2.0"   
    "FSharp.Data -Version 2.4.1" 
    "Newtonsoft.Json -Version 10.0.3"
  ]   
#load "Paket.Generated.Refs.fsx"

  
 
open System.IO
open System.Security.Cryptography
open Base58Check 

let toStream (s:string) = 
    let stream = new MemoryStream()
    let writer = new StreamWriter(stream)
    writer.Write(s)
    writer.Flush()
    stream.Position <- 0L
    stream

In [9]:
let SHA256 (content:string) = 
          let hashstring = new SHA256Managed()
          hashstring.ComputeHash(toStream(content))

let SHA256Encoded bytes = 
          Base58CheckEncoding.EncodePlain bytes

let SHA256EncodedPlain (content:string) = 
          Base58CheckEncoding.EncodePlain (SHA256 content)        

[Genesis Block] (http://api.qbit.ninja/blocks/000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f)

In [10]:
open FSharp.Data

type block = JsonProvider<"""
{
    "hash":"",
    "ver":0,
    "prev_block":"",
    "mrkl_root":"",
    "time":0,
    "bits":0,
    "nonce":0,
    "n_tx":0,
    "size":0,
    "block_index":0,
    "main_chain":true,
    "height":0,
    "received_time":0,
    "relayed_by":"",
    "tx":[]
}""">
      
let sblocks = block.GetSample()
  
let pool_uri = @"https://blockchain.info/it/rawblock/000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f"

let block =  block.Load(pool_uri)
 
 // Print general information
let rb = block.Hash 

printfn "%A" (block) 

{
  "hash": "000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f",
  "ver": 1,
  "prev_block": "0000000000000000000000000000000000000000000000000000000000000000",
  "mrkl_root": "4a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b",
  "time": 1231006505,
  "bits": 486604799,
  "fee": 0,
  "nonce": 2083236893,
  "n_tx": 1,
  "size": 285,
  "block_index": 14849,
  "main_chain": true,
  "height": 0,
  "tx": [
    {
      "lock_time": 0,
      "ver": 1,
      "size": 204,
      "inputs": [
        {
          "sequence": 4294967295,
          "witness": "",
          "script": "04ffff001d0104455468652054696d65732030332f4a616e2f32303039204368616e63656c6c6f72206f6e206272696e6b206f66207365636f6e64206261696c6f757420666f722062616e6b73"
        }
      ],
      "weight": 816,
      "time": 1231006505,
      "tx_index": 14849,
      "vin_sz": 1,
      "hash": "4a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b",
      "vout_sz": 1,
      "relayed_by": "0.0.

In [11]:

let previous_block_data = """0"""
let hashInit = (SHA256EncodedPlain previous_block_data)
printfn "Hash prev block init %A" hashInit

let data = """t(x1) + t(x2) + t(3)"""

printfn "%A" (SHA256 data)
let hashData = (SHA256EncodedPlain data)
printfn "%A" hashData

let sum = hashData + "|" + hashInit
let hashBlock = SHA256EncodedPlain sum


Hash prev block init "7TTGKXuhDL4XHeo2J2ZfKijhY4J8wYhPMHagzdUh6ZSQ"
[|41uy; 224uy; 22uy; 120uy; 114uy; 139uy; 110uy; 111uy; 136uy; 34uy; 190uy; 37uy;
  134uy; 177uy; 224uy; 193uy; 130uy; 85uy; 89uy; 33uy; 154uy; 242uy; 150uy;
  116uy; 18uy; 97uy; 179uy; 133uy; 48uy; 33uy; 43uy; 63uy|]
"3pTuCWUyJ44KXeDHJcJHRNNk1jbcz7eoraXkgS8gfTTt"


In [12]:
open System
open System.Collections.Generic

type Block = { PreviousHash:string; Data:string; Hash:string }

let genesis =  {PreviousHash = hashInit; Data = data; Hash = hashBlock }

// Create a new LinkedList.
let blockchain = new LinkedList<Block>()

let first_block= new LinkedListNode<Block>(genesis)

blockchain.AddLast(first_block)

printfn "%A" first_block

System.Collections.Generic.LinkedListNode`1[FSI_0027+Block]


In [13]:
let DisplayProperties (nodes:LinkedListNode<Block>) =
          if (nodes.List = null) then
             printfn "Node is not linked."
          else
             printfn "Node belongs to a linked list with %A elements."  nodes.List.Count
          if (nodes.Previous = null) then
             printfn "Previous node is null." 
          else
             printfn "Value of previous node: %A"  nodes.Previous.Value
          
          printfn "Value of current node:  \n%A"  nodes.Value     
          if (nodes.Next = null) then
             printfn "Next node is null."
          else
             printfn "Value of next node:     %A" nodes.Next.Value         
          0
      
DisplayProperties first_block

0

Node belongs to a linked list with 1 elements.
Previous node is null.
Value of current node:  
{PreviousHash = "7TTGKXuhDL4XHeo2J2ZfKijhY4J8wYhPMHagzdUh6ZSQ";
 Data = "t(x1) + t(x2) + t(3)";
 Hash = "FFpCGaPwPRpDoqjpHGuu5uaFigTgMU4DxuBENRxJDjTz";}
Next node is null.


In [14]:
blockchain |> Util.Table

PreviousHash,Data,Hash
7TTGKXuhDL4XHeo2J2ZfKijhY4J8wYhPMHagzdUh6ZSQ,t(x1) + t(x2) + t(3),FFpCGaPwPRpDoqjpHGuu5uaFigTgMU4DxuBENRxJDjTz


In [17]:
let data1 = """t(x1)' + t(x2)' + t(3)'"""

let sum1 = (SHA256EncodedPlain data1) + "|" + hashBlock
let hashBlock1 = SHA256EncodedPlain sum1

let one =  {PreviousHash = hashBlock; Data = data1; Hash = hashBlock1 }

let second_block = new LinkedListNode<Block>(one)
blockchain.AddLast(second_block)

printfn "%A" second_block


System.Collections.Generic.LinkedListNode`1[FSI_0027+Block]


In [19]:

//------------------------------------------------------

let data2 = """t(x1)'' + t(x2)'' + t(3)''"""

let sum2 = (SHA256EncodedPlain data2) + "|" + hashBlock1
let hashBlock2 = SHA256EncodedPlain sum2

let due =  {PreviousHash = hashBlock1; Data = data2; Hash = hashBlock2 }

let third_block = new LinkedListNode<Block>(due)
blockchain.AddLast(third_block)


In [9]:
//------------------------------------------------------
blockchain |> Util.Table


PreviousHash,Data,Hash
7TTGKXuhDL4XHeo2J2ZfKijhY4J8wYhPMHagzdUh6ZSQ,t(x1) + t(x2) + t(3),FFpCGaPwPRpDoqjpHGuu5uaFigTgMU4DxuBENRxJDjTz
FFpCGaPwPRpDoqjpHGuu5uaFigTgMU4DxuBENRxJDjTz,t(x1)' + t(x2)' + t(3)',7DYwN5hHovwwHDdWf77eP1PX6z235294H74p3B6ZQfNt
7DYwN5hHovwwHDdWf77eP1PX6z235294H74p3B6ZQfNt,t(x1)'' + t(x2)'' + t(3)'',9eXUMhKXzDPa27GDDPaomehg46vLK2XBfgRPzJrjLdc3


In [20]:
// Move the first node to be the last node.
let genesis_bis:LinkedListNode<Block> = blockchain.First
blockchain.RemoveFirst()
blockchain.AddLast(genesis_bis)

blockchain |> Util.Table

PreviousHash,Data,Hash
FFpCGaPwPRpDoqjpHGuu5uaFigTgMU4DxuBENRxJDjTz,t(x1)' + t(x2)' + t(3)',7DYwN5hHovwwHDdWf77eP1PX6z235294H74p3B6ZQfNt
7DYwN5hHovwwHDdWf77eP1PX6z235294H74p3B6ZQfNt,t(x1)'' + t(x2)'' + t(3)'',9eXUMhKXzDPa27GDDPaomehg46vLK2XBfgRPzJrjLdc3
FFpCGaPwPRpDoqjpHGuu5uaFigTgMU4DxuBENRxJDjTz,t(x1)' + t(x2)' + t(3)',7DYwN5hHovwwHDdWf77eP1PX6z235294H74p3B6ZQfNt
7DYwN5hHovwwHDdWf77eP1PX6z235294H74p3B6ZQfNt,t(x1)'' + t(x2)'' + t(3)'',9eXUMhKXzDPa27GDDPaomehg46vLK2XBfgRPzJrjLdc3
7DYwN5hHovwwHDdWf77eP1PX6z235294H74p3B6ZQfNt,t(x1)'' + t(x2)'' + t(3)'',9eXUMhKXzDPa27GDDPaomehg46vLK2XBfgRPzJrjLdc3
7TTGKXuhDL4XHeo2J2ZfKijhY4J8wYhPMHagzdUh6ZSQ,t(x1) + t(x2) + t(3),FFpCGaPwPRpDoqjpHGuu5uaFigTgMU4DxuBENRxJDjTz
